## 1.导入模块

In [1]:
import moxing as mox

data_url = "s3://nlp.final/san/data/"
code_url = "s3://nlp.final/san/code/image_model/src"
# mox.file.copy_parallel(src_url="obs://nlp-kim/project/data/", dst_url='./data/') 
# mox.file.copy_parallel(src_url="s3://dl4nlp-my/project/data/", dst_url='./data/') 
mox.file.copy_parallel(src_url = code_url,dst_url='./src/')

INFO:root:Using MoXing-v1.17.3-d858ff4a
INFO:root:Using OBS-Python-SDK-3.20.9.1


In [2]:
mox.file.copy_parallel(src_url=data_url, dst_url='./data/') 

INFO:root:Listing OBS: 1000
INFO:root:Listing OBS: 2000
INFO:root:Listing OBS: 3000
INFO:root:Listing OBS: 4000
INFO:root:Listing OBS: 5000
INFO:root:Listing OBS: 6000
INFO:root:Listing OBS: 7000
INFO:root:Listing OBS: 8000
INFO:root:Listing OBS: 9000
INFO:root:Listing OBS: 10000
INFO:root:Listing OBS: 11000
INFO:root:Listing OBS: 12000
INFO:root:Listing OBS: 13000
INFO:root:Listing OBS: 14000
INFO:root:Listing OBS: 15000
INFO:root:Listing OBS: 16000
INFO:root:Listing OBS: 17000
INFO:root:Listing OBS: 18000
INFO:root:Listing OBS: 19000
INFO:root:Listing OBS: 20000
INFO:root:Listing OBS: 21000
INFO:root:Listing OBS: 22000
INFO:root:Listing OBS: 23000
INFO:root:Listing OBS: 24000
INFO:root:Listing OBS: 25000
INFO:root:Listing OBS: 26000
INFO:root:Listing OBS: 27000
INFO:root:Listing OBS: 28000
INFO:root:Listing OBS: 29000
INFO:root:Listing OBS: 30000
INFO:root:Listing OBS: 31000
INFO:root:Listing OBS: 32000
INFO:root:Listing OBS: 33000
INFO:root:Listing OBS: 34000
INFO:root:Listing OBS: 

In [3]:
import sys
import os
import json
import pickle as pkl
import re
from collections import Counter
import numpy as np
import random

from collections import OrderedDict
import math

import mindspore
import mindspore.nn as nn
from mindspore import Tensor,Model
from mindspore import context
from mindspore.train.model import Model
from mindspore.nn.metrics import Accuracy
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor, TimeMonitor
#from mindspore.ops import operations as ops
from mindspore import ops
from mindspore import dtype as mstype

## 2.数据预处理

### 2.1 处理函数及初始化函数

In [4]:
# mox.file.copy_parallel(src_url="obs://nlp-kim/project/code/data_vqa/process_func.py", dst_url='./process_func.py')
# mox.file.copy_parallel(src_url="obs://nlp-kim/project/code/data_vqa/initialization.py", dst_url='./initialization.py')
mox.file.copy_parallel(src_url = "s3://nlp.final/san/code/data_vqa",dst_url="./data_vqa")
from data_vqa.process_func import *
#from data_vqa.initialization import *
from data_vqa.vqa_dataset_by_api import create_dataset
from data_vqa.vqa_config import train,test,val

### 2.2 变量说明

In [5]:
#qa：question和对应的annotions
#train_question_ids：question的id的数组

#question_dict_count： question中的单词出现次数统计
#train_questions： question语句split为word的数组的数组
#answer_dict_count： answer中的单词出现次数统计
#train_answers： answer被split为word的数组的数组

#question_key：按照question中出现次数进行排序
#answer_top_k: 按照answer中出现的次数进行排序


### 2.3 预处理细节

In [6]:
f = open("./data/questions/train.json", "r")
f1 = open("./data/annotations/train.json", "r")
file = json.load(f)
file1 = json.load(f1)
annotations = file1['annotations']
train_question_ids = []
train_image_ids = []
train_questions = []
train_answers = []
question_dict_count = dict()
answer_dict_count = dict()

# 形成qa：一个字典，整理出对应question_id的annotation
qa = {ann['question_id']: [] for ann in annotations}
for ann in annotations:
    qa[ann['question_id']] = ann

#获取image_id question_id
# 需要进行封装以适用于训练、测试、评估
img_not_exist_count = 0
for idx, item in enumerate(file['questions']):
    img_path = os.path.join("./data/images/train","COCO_{0}2014_{1}.jpg".format("train",str(item['image_id']).zfill(12)))
    if not os.path.exists(img_path):
        img_not_exist_count += 1
        continue

    train_question_ids.append(item['question_id'])
    train_image_ids.append(item['image_id'])
    
    #process question
    question = item['question']
    question = process_sentence(question)
    question = question.split()
    for word in question:
        question_dict_count[word] = question_dict_count.get(word, 0) + 1
    train_questions.append(question)
    answer = qa[item['question_id']]['answers']
    answer_new = [process_answer(ans['answer']) for ans in answer]
    ans_array = []
    for ans in answer:
        ans_array.append(ans['answer'])
    for word in answer_new:
        answer_dict_count[word] = answer_dict_count.get(word, 0) + 1
    train_answers.append(ans_array)
    if idx % 10000 == 0:
        print ('finished processing %d in train' %(idx))

# sort question dict
question_count = question_dict_count.values()
sorted_index = [count[0] for count in
                sorted(enumerate(question_count),
                       key = lambda x : x[1],
                       reverse=True)]
sorted_count = sorted(question_count, reverse=True)
question_key = list(question_dict_count.keys())
# 对question_key重新排序
question_key = [question_key[idx] for idx in sorted_index]
# add '<unk>' to the begining
question_key.insert(0, '<unk>')
# '<unk>' begins at 1, 0 is reserved for empty words
question_key = dict((key, idx + 1) for idx, key in enumerate(question_key))

k = 1000
# sort answer dict and get top k answers

#因报错暂删
# del answer_dict_count['']
answer_count = answer_dict_count.values()
sorted_index = [count[0] for count in
                sorted(enumerate(answer_count),
                       key = lambda x : x[1],
                       reverse=True)]
sorted_count = sorted(answer_count, reverse=True)
answer_key = list(answer_dict_count.keys())
answer_key = [answer_key[idx] for idx in sorted_index]
answer_top_k = answer_key[:k]
answer_top_k = dict((key, idx) for idx, key in enumerate(answer_top_k))

# convert words to idx and remove some
train_question_idx = []
train_answer_idx = []
train_answer_counter = []
idx_to_remove = []
for idx, answer in enumerate(train_answers):
    question_idx = [question_key[word] for word in train_questions[idx]]
    #print(question_idx)
    #print('\n')
    #print(train_questions[idx])
    train_question_idx.append(question_idx)
    answer_idx = [answer_top_k[ans] for ans in answer
                 if ans in answer_top_k]
    answer_counter = Counter(answer_idx)
    train_answer_counter.append(answer_counter)
    train_answer_idx.append(answer_idx)
    if not answer_idx:
        idx_to_remove.append(idx)
print ('%d out of %d, %f of the question in train are removed'\
    %(len(idx_to_remove), len(train_question_ids),
      len(idx_to_remove) / float(len(train_question_ids))))

# transform to array and delete all the empty answer
train_question_ids = np.array(train_question_ids)
train_image_ids = np.array(train_image_ids)
train_question_idx = np.array(train_question_idx)
train_answer_idx = np.array(train_answer_idx)
train_answer_counter = np.array(train_answer_counter)

train_question_ids = np.delete(train_question_ids, idx_to_remove)
train_image_ids = np.delete(train_image_ids, idx_to_remove)
train_question_idx = np.delete(train_question_idx, idx_to_remove)
train_answer_idx = np.delete(train_answer_idx, idx_to_remove)
train_answer_counter = np.delete(train_answer_counter, idx_to_remove)

# reshuffle the train data
idx_shuffle = list(range(train_question_ids.shape[0]))
random.shuffle(idx_shuffle)
train_question_ids = train_question_ids[idx_shuffle]
train_image_ids = train_image_ids[idx_shuffle]
train_question_idx = train_question_idx[idx_shuffle]
train_answer_idx = train_answer_idx[idx_shuffle]
train_answer_counter = train_answer_counter[idx_shuffle]

# the most frequent as label
train_answer_label = [counter.most_common(1)[0][0]
                      for counter in train_answer_counter]
train_answer_label = np.array(train_answer_label)

# transform from counter to dict
train_answer_counter = [dict(counter) for counter in train_answer_counter]
train_answer_counter = np.array(train_answer_counter)

print ('finished processing train')

print('{0} images not exist'.format(img_not_exist_count))

finished processing 10000 in train
finished processing 20000 in train
950 out of 23071, 0.041177 of the question in train are removed
finished processing train
21304 images not exist


In [7]:
print(answer_top_k)

{'no': 0, 'yes': 1, '1': 2, '2': 3, 'white': 4, '0': 5, '3': 6, 'black': 7, 'red': 8, 'blue': 9, '4': 10, 'brown': 11, 'green': 12, 'yellow': 13, '5': 14, 'gray': 15, 'nothing': 16, '6': 17, 'baseball': 18, 'frisbee': 19, 'pizza': 20, 'right': 21, 'kitchen': 22, 'tennis': 23, 'left': 24, 'wood': 25, 'bathroom': 26, 'orange': 27, '7': 28, 'pink': 29, '8': 30, 'living room': 31, 'dog': 32, 'man': 33, 'cat': 34, 'phone': 35, 'silver': 36, 'skateboarding': 37, '10': 38, 'grass': 39, 'kite': 40, 'water': 41, 'cake': 42, 'table': 43, 'surfing': 44, 'skateboard': 45, 'tan': 46, 'skiing': 47, 'black and white': 48, 'snow': 49, 'wii': 50, 'standing': 51, 'surfboard': 52, '9': 53, 'cow': 54, 'laptop': 55, 'bus': 56, 'giraffe': 57, 'snowboarding': 58, 'woman': 59, 'purple': 60, 'elephant': 61, 'horse': 62, 'motorcycle': 63, 'soccer': 64, 'stop': 65, 'apple': 66, 'food': 67, 'banana': 68, 'sunny': 69, 'male': 70, 'beach': 71, '20': 72, 'eating': 73, 'clear': 74, 'unknown': 75, 'maybe': 76, 'bear':

In [58]:
import math
import os
import json
import numpy as np


import mindspore.dataset.vision.py_transforms as vision
import mindspore.dataset as de
from PIL import Image
from easydict import EasyDict as edict

def img2tensor(img):
    # mean = [0.485 * 255, 0.456 * 255, 0.406 * 255]
    # std = [0.229 * 255, 0.224 * 255, 0.225 * 255]
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    transform = edict({
        "ToPIL": vision.ToPIL(),
        "Decode": vision.Decode(),
        "Resize": vision.Resize((512, 512)),
        "CenterCrop": vision.CenterCrop(448),
        "ToTensor":vision.ToTensor(),
        # "Rescale":vision.Rescale(1.0/255.0,0.0),
        "Normalize": vision.Normalize(mean=mean, std=std),
        "HWC2CHW": vision.HWC2CHW(),
    })
    # print("input image:",img.shape)
    img = transform.HWC2CHW(img) / 255
    # print("before normaize",img)
    # img = (img - mean[:, None, None]) / std[:, None, None]
    img = transform.Normalize(img)
    # print('img after normalize:',img)
    # print("totensor:",img)
    # img = [img]
    # tensor = Tensor(img,mstype.float32)
    # return tensor
    return img
    
def create_dataset(batch_size,mode = 'train',drop_remainder=True,q_dict = None,a_dict=None):
    dataset = VQADataset(mode,q_dict = q_dict,a_dict = a_dict)
    sampler = DistributedSampler(dataset)

    de_dataset = de.GeneratorDataset(dataset, ["image", "question","label"],shuffle=False,sampler=sampler)
    de_dataset = de_dataset.map(operations=img2tensor, input_columns="image", num_parallel_workers=8)
    # de_dataset = de_dataset.map(operations=None, input_columns="question", num_parallel_workers=8)
    # de_dataset = de_dataset.map(operations=None, input_columns="label", num_parallel_workers=8)
    de_dataset = de_dataset.project(columns=['image','question','label'])
    de_dataset = de_dataset.batch(batch_size, drop_remainder=drop_remainder)
    
    return de_dataset
    
class DistributedSampler():
    """
    sampling the dataset.
    Args:
    Returns:
        num_samples, number of samples.
    """
    def __init__(self, dataset, rank=0, group_size=1, shuffle=True, seed=0):
        self.dataset = dataset
        self.rank = rank
        self.group_size = group_size
        self.dataset_length = len(self.dataset)
        self.num_samples = int(math.ceil(self.dataset_length * 1.0 / self.group_size))
        self.total_size = self.num_samples * self.group_size
        self.shuffle = shuffle
        self.seed = seed

    def __iter__(self):
        if self.shuffle:
            self.seed = (self.seed + 1) & 0xffffffff
            np.random.seed(self.seed)
            indices = np.random.permutation(self.dataset_length).tolist()
        else:
            indices = list(range(len(self.dataset_length)))

        indices += indices[:(self.total_size - len(indices))]
        indices = indices[self.rank::self.group_size]
        return iter(indices)

    def __len__(self):
        return self.num_samples

class VQADataset:
    """
    Args:
    mode: train/val/test
    cfg: 
    Returns:
        de_dataset.
    """
    def __init__(self, mode = "train",q_dict =None,a_dict = None):
        super(VQADataset, self).__init__()
        self.Resize =  vision.Resize((512, 512))
        self.CenterCrop =  vision.CenterCrop(448)
        self.images = []   # image paths
        self.questions = []   #  questions 应该是已经转换成one-hot的编码
        self.answers = []   # answers   对应的正确答案
        self.q_dict = q_dict
        self.a_dict = a_dict
        ann = {}
        ques = {}
        image_path = ''
        if mode == "train":
            ann = json.load(open(train.annotation,'r'))
            ques = json.load(open(train.question,'r'))
            image_path = train.image
        elif mode == 'test':
            ann = json.load(open(test.annotation,'r'))
            ques = json.load(open(test.question,'r'))
            image_path = test.image
        else:
            ann = json.load(open(val.annotation,'r'))
            ques = json.load(open(val.question,'r'))
            image_path = val.image
        
        self.question_dict = {item['question_id']:item['question'] for item in ques['questions']}
        annotations = ann['annotations']
        img_not_exist_count = 0
        for index,item in enumerate(annotations):
            img_path = os.path.join(image_path,"COCO_{0}2014_{1}.jpg".format(mode,str(item['image_id']).zfill(12)))
            if not os.path.exists(img_path):  # filter non-existing image
                img_not_exist_count += 1
                continue
            
            answers = item['answers']     # filter blank answer
            if len(answers) == 0:
                continue
            
            election = dict() 
            for index,ans in enumerate(answers):
                if ans['answer_confidence'] == 'yes':
                    election[ans['answer']] = election.get(ans['answer'],0)+1
            if not election:
                continue
                
            elected = max(election, key=election.get)
            if not elected in self.a_dict:
                continue
            
            if mode == 'val':
                sentence = self.question_dict[item['question_id']]
                sentence = process_sentence(sentence)
                sentence = sentence.split()
                x = 0
                for word in sentence:
                    if not word in self.q_dict:
                        x = 1
                        break;
                if x == 1:
                    continue

            self.answers.append(elected)
            self.images.append(img_path)
            self.questions.append(item['question_id'])
            

    def __getitem__(self, index):
        image = Image.open(self.images[index]).convert('RGB')
        image = self.Resize(image)
        image = self.CenterCrop(image)
        # image = self.images[index]

        question_str = self.question_dict[self.questions[index]]
        question = process_sentence(question_str)
        question = question.split()
        question = convert_sentence_to_vec(question,self.q_dict)

#         answer = self.a_dict[self.answers[index]]
        answer = np.zeros(1000,dtype=np.float32)
        answer[self.a_dict[self.answers[index]]] = 1
        return image, question,answer

    def __len__(self):
        return len(self.questions)


### 2.4 构建词向量

In [9]:
# #不用跑这段
# #construct one hot vector
# all_question_vector=[]
# for idx,question in enumerate(train_questions):
#     count = 0
#     question_vector = []
#     for word in question:
#         count = count + 1
#         if count > 10:
#             break
#         else:
#             q_emb = np.zeros((len(question_key) + 1), dtype='int32')
#             q_emb[question_key[word]] = 1
#             question_vector.append(q_emb)
#     while count < 10:
#         padding = np.zeros((len(question_key) + 1), dtype='int32')
#         question_vector.append(padding)
#         count = count + 1
#     all_question_vector.append(question_vector)

In [9]:
#convert word to idx
all_question_idx = []
for question in train_questions:
    all_question_idx.append(convert_sentence_to_vec(question, question_key))
all_question_idx

[[5, 2, 242, 1035, 7, 10, 2857, 13, 0, 0],
 [4, 3, 2, 16, 31, 0, 0, 0, 0, 0],
 [4, 3, 2, 28, 20, 0, 0, 0, 0, 0],
 [148, 162, 2134, 46, 447, 7, 2, 196, 0, 0],
 [22, 3, 2, 166, 0, 0, 0, 0, 0, 0],
 [3, 2, 61, 288, 259, 0, 0, 0, 0, 0],
 [4, 13, 136, 3, 1490, 19, 2, 212, 0, 0],
 [5, 34, 65, 2858, 0, 0, 0, 0, 0, 0],
 [4, 13, 3, 2, 37, 0, 0, 0, 0, 0],
 [4, 5, 2, 739, 53, 8, 0, 0, 0, 0],
 [4, 26, 9, 47, 3, 69, 0, 0, 0, 0],
 [4, 3, 2, 740, 8, 2, 255, 42, 0, 0],
 [4, 740, 3, 8, 2, 255, 42, 0, 0, 0],
 [3, 6, 708, 0, 0, 0, 0, 0, 0, 0],
 [11, 12, 2859, 5, 14, 0, 0, 0, 0, 0],
 [87, 162, 1109, 94, 0, 0, 0, 0, 0, 0],
 [3, 14, 10, 1734, 7, 101, 9, 2, 114, 0],
 [3, 2, 272, 649, 0, 0, 0, 0, 0, 0],
 [3, 2, 115, 470, 121, 2135, 0, 0, 0, 0],
 [4, 13, 3, 2, 58, 134, 0, 0, 0, 0],
 [3, 2, 44, 779, 7, 6, 18, 897, 0, 0],
 [4, 13, 5, 2, 2860, 7, 2, 1036, 0, 0],
 [3, 6, 10, 117, 250, 92, 0, 0, 0, 0],
 [11, 12, 1735, 15, 6, 44, 25, 0, 0, 0],
 [4, 13, 3, 2, 63, 0, 0, 0, 0, 0],
 [4, 3, 2, 1212, 16, 2861, 19, 2, 1212,

## 3.训练过程

### 3.1 超参数设置

In [10]:
#from easydict import EasyDict as edict
options = OrderedDict()
# data related
options['data_path'] = './data/'
#options['feature_file'] = 'trainval_feat.h5'
#options['expt_folder'] = 'expt_1'
options['model_name'] = 'imageqa'
options['train_split'] = 'trainval1'
options['val_split'] = 'val2'
options['shuffle'] = True
options['reverse'] = True
options['sample_answer'] = True

options['num_region'] = 196
options['region_dim'] = 512

#`13746
options['n_words'] = 6620
options['n_output'] = 1000

# structure options
options['combined_num_mlp'] = 1
options['combined_mlp_drop_0'] = True
options['combined_mlp_act_0'] = 'linear'
options['sent_drop'] = False
options['use_tanh'] = False

options['use_attention_drop'] = False

# dimensions
options['n_emb'] = 500
options['n_dim'] = 1024
options['n_image_feat'] = options['region_dim']
options['n_common_feat'] = 500
options['n_attention'] = 512

# initialization
options['init_type'] = 'uniform'
options['range'] = 0.01
options['std'] = 0.01
options['init_lstm_svd'] = False

options['forget_bias'] = 1.0

# learning parameters
options['optimization'] = 'sgd' # choices
options['batch_size'] = 32
options['lr'] = 0.05
options['w_emb_lr'] = 80
options['momentum'] = 0.9
options['gamma'] = 1
options['step'] = 10
options['step_start'] = 100
options['max_epochs'] = 50
options['weight_decay'] = 0.0005
options['decay_rate'] = 0.999
options['drop_ratio'] = 0.5
options['smooth'] = 1e-8
options['grad_clip'] = 0.1

# log params
options['disp_interval'] = 10
options['eval_interval'] = 1000
options['save_interval'] = 500

#new
options['dict_size'] = 6620

In [11]:
# vgg模型导入相关包
# mox.file.copy_parallel(src_url="obs://nlp-kim/project/code/image_model/src",dst_url="./src/")
from PIL import Image, ImageFile
from src.utils.logging import get_logger
from src.dataset import classification_dataset
from easydict import EasyDict as edict
from src.config import imagenet_cfg as vgg_cfg
# from data_vqa.vqa_config import model_cfg
from src.vgg import vgg16

import time
import datetime
import mindspore.dataset.vision.py_transforms as vision

In [12]:
model_cfg = edict({
    "cnn_ckpt_19":"./data/vgg19_ascend_v111_imagenet2012_research_cv_bs64_acc74.ckpt",
    "cnn_ckpt_16":"./data/vgg16_ascend_v120_imagenet2012_official_cv_bs32_acc73.ckpt",
    "log_path":"./outputs",

    "device_target": 'Ascend',
    "per_batch_size": 32,
    "graph_ckpt":1,
    "rank": 0,
    "group_size":1
})

In [13]:
image_path = {
    "train":"./data/images/train",
    "test":"./data/images/test",
    "val":"./data/images/val"
}

annotation_path = {
    "train":"./data/annotation/train.json",
    "test":"./data/annotation/test.json",
    "val":"./data/annotation/val.json"
}

model_cfg= edict({
    "annotation":annotation_path,
    "image":image_path,
    "cnn_ckpt_19":"./data/vgg19_ascend_v111_imagenet2012_research_cv_bs64_acc74.ckpt",
    "cnn_ckpt_16":"./data/vgg16_ascend_v120_imagenet2012_official_cv_bs32_acc73.ckpt",
    "log_path":"./outputs",

    "device_target": 'Ascend',
    "per_batch_size": 32,
    "graph_ckpt":1,
    "rank": 0,
    "group_size":1
})

In [14]:
context.set_context(mode=context.GRAPH_MODE, device_target=model_cfg.device_target, device_id=0,enable_auto_mixed_precision=True,save_graphs=False)

In [15]:
model_cfg.outputs_dir = os.path.join(model_cfg.log_path,
                                    datetime.datetime.now().strftime('%Y-%m-%d_time_%H_%M_%S'))

model_cfg.logger = get_logger(model_cfg.outputs_dir, model_cfg.rank)

### 3.2模型搭建

In [16]:
# 导入vgg模型
model_cfg.logger.important_info('start create vgg')
vgg = vgg16(vgg_cfg.num_classes, vgg_cfg, phase="test",include_top = False)
vgg.add_flags_recursive(fp32=True)
vgg.set_train(False)
model_cfg.logger.important_info('start load checkpoint')
param_dict = load_checkpoint(model_cfg.cnn_ckpt_16)
load_param_into_net(vgg, param_dict)

2021-07-15 06:44:53,289:INFO:
**********************************************************************
**********************************************************************
**
**
**        start create vgg
**
**
**********************************************************************
**********************************************************************

2021-07-15 06:44:53,851:INFO:
**********************************************************************
**********************************************************************
**
**
**        start load checkpoint
**
**
**********************************************************************
**********************************************************************



[]

In [17]:
# def img2tensor(img_root,img_prefix,img_id):
#         img_path = os.path.join(img_root,img_prefix+"_"+str(img_id).zfill(12)+".jpg")
#         print("img_path:",img_path)
#         img = Image.open(img_path).convert('RGB')
#         transform = edict({
# #             "Decode": vision.Decode(),
#             "Resize": vision.Resize((512, 512)),
#             "CenterCrop": vision.CenterCrop(448),
# #             "Normalize": vision.Normalize(mean=mean, std=std),
# #             "HWC2CHW": vision.HWC2CHW(),
#             "ToTensor":vision.ToTensor()
#         })
#         # img = transform.Decode(img)
#         img = transform.Resize(img)
#         img = transform.CenterCrop(img)
#         img = transform.ToTensor(img)
#         print("totensor:",img)
#     #     img = transform.Normalize(img) # CHW
#     #     print("normalized:",img)
#     #     img = transform.HWC2CHW(img)
#         img = [img]
#         tensor = Tensor(img,mstype.float32)
#         return tensor

In [18]:
def get_feature_map(tensor):
    model_cfg.logger.important_info('start get feature map')
    output = vgg(tensor)  
    return output

In [19]:
# img = img2tensor(model_cfg.image.train,"COCO_train2014",9)
# image_feat = get_feature_map(img)

In [20]:
# image_feat = image_feat.transpose(0,2,3,1)
# image_feat = image_feat.reshape(1, 196, 512)

In [21]:
from collections import Counter
import numpy as np
import random
from collections import OrderedDict
import math

import mindspore
import mindspore.nn as nn
from mindspore import Tensor

#from mindspore.ops import operations as ops
from mindspore import ops
from mindspore import dtype as mstype

floatX = np.float32

def init_weight(n, d, options):
    ''' initialize weight matrix
    options['init_type'] determines
    gaussian or uniform initlizaiton
    '''
    if options['init_type'] == 'gaussian':
        return Tensor((np.random.randn(n, d)).astype(floatX)) * options['std']
    elif options['init_type'] == 'uniform':
        # [-range, range]
        return Tensor(((np.random.rand(n, d) * 2 - 1) * \
                options['range']).astype(floatX))
    
def ortho_weight(ndim):
    """
    Random orthogonal weights, we take
    the right matrix in the SVD.

    Remember in SVD, u has the same # rows as W
    and v has the same # of cols as W. So we
    are ensuring that the rows are
    orthogonal.
    """
    W = np.random.randn(ndim, ndim)
    u, _, _ = np.linalg.svd(W)
    return Tensor(u).astype('float32')

def init_fflayer(params, nin, nout, options, prefix='ff'):
    ''' initialize ff layer
    '''
    params[prefix + '_w'] = init_weight(nin, nout, options)
    params[prefix + '_b'] = Tensor(np.zeros(nout, dtype='float32'))
    return params

# def init_lstm_layer(params, nin, ndim, options, prefix='lstm'):
#     ''' initializt lstm layer
#     '''
#     params[prefix + '_w_x'] = init_weight(nin, 4 * ndim, options)
#     # use svd trick to initializ
#     if options['init_lstm_svd']:
#         params[prefix + '_w_h'] = Tensor(np.concatenate([ortho_weight(ndim),
#                                                   ortho_weight(ndim),
#                                                   ortho_weight(ndim),
#                                                   ortho_weight(ndim)],
#                                                  axis=1))
#     else:
#         params[prefix + '_w_h'] = init_weight(ndim, 4 * ndim, options)
#     params[prefix + '_b_h'] = Tensor(np.zeros(4 * ndim, dtype='float32'))
#     # set forget bias to be positive
#     params[prefix + '_b_h'][ndim : 2*ndim] = Tensor(np.float32(options.get('forget_bias', 0)))
#     return params

# initialize the parmaters
def init_params(options):
    ''' Initialize all the parameters
    '''
    params = OrderedDict()
    n_words = options['n_words']
    n_emb = options['n_emb']
    n_dim = options['n_dim']
    n_image_feat = options['n_image_feat']
    n_common_feat = options['n_common_feat']
    n_output = options['n_output']
    n_attention = options['n_attention']

    params['w_emb'] = Tensor((np.random.rand(n_words, n_emb) * 2 - 1) * 0.5).astype(floatX)

    params = init_fflayer(params, n_image_feat, n_dim, options,
                          prefix='image_mlp')

    # attention model based parameters
    params = init_fflayer(params, n_dim, n_attention, options,
                          prefix='image_att_mlp_1')
    params = init_fflayer(params, n_dim, n_attention, options,
                          prefix='sent_att_mlp_1')
    params = init_fflayer(params, n_attention, 1, options,
                          prefix='combined_att_mlp_1')
    params = init_fflayer(params, n_dim, n_attention, options,
                          prefix='image_att_mlp_2')
    params = init_fflayer(params, n_dim, n_attention, options,
                          prefix='sent_att_mlp_2')
    params = init_fflayer(params, n_attention, 1, options,
                          prefix='combined_att_mlp_2')


    # params for sentence image mlp
    for i in range(options['combined_num_mlp']):
        if i == 0 and options['combined_num_mlp'] == 1:
            params = init_fflayer(params, n_dim, n_output,
                                  options, prefix='combined_mlp_%d'%(i))
        elif i == 0 and options['combined_num_mlp'] != 1:
            params = init_fflayer(params, n_dim, n_common_feat,
                                  options, prefix='combined_mlp_%d'%(i))
        elif i == options['combined_num_mlp'] - 1 :
            params = init_fflayer(params, n_common_feat, n_output,
                                  options, prefix='combined_mlp_%d'%(i))
        else:
            params = init_fflayer(params, n_common_feat, n_common_feat,
                                  options, prefix='combined_mlp_%d'%(i))

    # lstm layer
    #params = init_lstm_layer(params, n_emb, n_dim, options, prefix='sent_lstm')

    return params

def init_shared_params(params):
    ''' return a shared version of all parameters
    '''
    global shared_params
    shared_params = OrderedDict()
    for k, p in params.items():
        shared_params[k] = params[k]

    return shared_params

def fflayer(shared_params, x, options, prefix='ff', act_func='tanh'):
    ''' fflayer: multiply weight then add bias
    '''
    tanh = nn.Tanh()
    input = ops.dot(x, Tensor(shared_params[prefix + '_w'])) + \
                          Tensor(shared_params[prefix + '_b'])
    return tanh(input)

def fflayer1(shared_params, x, options, prefix='ff'):
    ''' fflayer: multiply weight then add bias
    '''
    input = ops.dot(x, Tensor(shared_params[prefix + '_w'])) + \
                          Tensor(shared_params[prefix + '_b'])
    return input

def get_lr(options, curr_epoch):
    if options['optimization'] == 'sgd':
        power = max((curr_epoch - options['step_start']) / options['step'], 0)
        power = math.ceil(power)
        return options['lr'] * (options['gamma'] ** power)  #
    else:
        return options['lr']

In [22]:
floatX = np.float32
batch_size = options['batch_size']
max_epochs = options['max_epochs']

###############
# build model #
###############
params = init_params(options)
shared_params = init_shared_params(params)

In [23]:
# input_idx = np.ones((6618,100),dtype = 'int32')
# shared_params['w_emb'] = ((np.random.rand(6620, 500) * 2 - 1) * 0.5).astype(floatX)
# empty_word = np.zeros((1, 500), dtype='float32')
# w_emb_extend = shared_params['w_emb']
# input_emb = w_emb_extend[input_idx]

In [24]:
class LSTM(nn.Cell):
    def __init__(self, options, is_training=True):
        super(LSTM, self).__init__()
        if is_training:
            self.batch_size = options['batch_size']
        else:
            self.batch_size = 1
            
        self.n_dim = options['n_dim']
        self.n_emb = options['n_emb']
        self.dropout = options['drop_ratio']
        
        # TODO
        self.h = Tensor(np.zeros((1,self.batch_size, self.n_dim), dtype='float32'))
        self.c = Tensor(np.zeros((1,self.batch_size, self.n_dim), dtype='float32'))
        
        self.rnn = nn.LSTM(self.n_emb,self.n_dim,1,True,True,self.dropout)
        #self.cast = P.Cast()

    def construct(self, x):
        #x = self.cast(x, mstype.float16)
        output,(h1,c1) = self.rnn(x, (self.h,self.c))
        return output,(h1,c1)

class Question(nn.Cell):
    def __init__(self, options, is_training=True):
        super(Question, self).__init__()
        #dict_size(vocab_size)
        self.dict_size = options['dict_size']
        #n_dim (hidden_size)
        self.n_dim = options['n_dim']
        self.n_emb = options['n_emb']
        
        if is_training:
            self.batch_size = options['batch_size']
        else:
            self.batch_size = 1

        #self.trans = P.Transpose()
        #self.perm = (1, 0, 2)
        
        #HIGHLIGHT 第二个参数n_dim -> n_emb
        self.embedding = nn.Embedding(self.dict_size, self.n_emb)
        #?
        self.lstm = LSTM(options, is_training=is_training).to_float(mstype.float16)
        #self.h = Tensor(np.zeros((self.batch_size, self.n_dim)).astype(np.float16))
        #self.c = Tensor(np.zeros((self.batch_size, self.n_dim)).astype(np.float16))

    def construct(self, question_input):
        embeddings = self.embedding(question_input)
        #embeddings = self.trans(embeddings, self.perm)
        output, (hn,cn) = self.lstm(embeddings)
        return output, hn, cn


In [25]:
from mindspore.ops import operations as P

In [50]:
class VQA(nn.Cell):
    def __init__(self, options, is_train=True):
        super(VQA, self).__init__()
        #         self.max_len = config.max_seq_length
        self.is_train = is_train
        self.batch_size = options['batch_size']
        #self.encoder = Encoder(config, is_train)
        #self.decoder = Decoder(config, is_train)
        self.expanddims = P.ExpandDims()
        self.squeeze = P.Squeeze(axis=0)
        self.argmax = P.ArgMaxWithValue(axis=int(2), keep_dims=True)
        self.concat = P.Concat(axis=1)
        self.concat2 = P.Concat(axis=0)
        self.select = P.Select()
        self.softmax = nn.Softmax()
        self.print = P.Print()
        self.dense1 = nn.Dense(1024,1000)
        self.dense2 = nn.Dense(512,1024)
        self.mul = ops.Mul()
        
        
        ##### my #####
        self.question = Question(options, is_train)
        self.vgg = vgg

    def construct(self, x, src):
        self.print("enter construct")
        #         self.print("x:",type(x))
        image_feat = self.vgg(x)
        image_feat = image_feat.transpose(0,2,3,1)
        image_feat = image_feat.reshape(self.batch_size,196,512)
        image_feat = image_feat.mean(axis = (1),keep_dims = False)
        image_feat = self.dense2(image_feat)
        
        output,h_encode,c_encode = self.question(src)

        h_encode = h_encode[0]
        #         h_encode = h_encode[None,:]
        # h_encode.shape 1,1024
        output = self.mul(h_encode,image_feat)
        output = self.dense1(output)
        prob = self.softmax(output)
        return prob
#         image_feat = fflayer(shared_params, image_feat, options,
#                              prefix='image_mlp',
#                               act_func='tanh')
#         image_feat = image_feat[0]
#         #image_feat.shape 196,1024
        
#         image_feat_down = image_feat
#         #image_feat_down.shape 196,1024
        
#         image_feat_attention_1 = fflayer(shared_params, image_feat_down, options,
#                                          prefix='image_att_mlp_1',
#                                          act_func='tanh')
#         #image_feat_attention_1.shape 196,512

        
#         h_encode_attention_1 = fflayer(shared_params, h_encode, options,
#                                        prefix='sent_att_mlp_1',
#                                        act_func='tanh')
#         #h_encode_attentioshapehape 1,512
        
#         combined_feat_attention_1 = image_feat_attention_1 + \
#                             h_encode_attention_1[:, None, :]
#         #combined_feat_attention_1.shape 1,196,512

#         #if options['use_attention_drop']:
#         #combined_feat_attention_1 = dropout_layer(combined_feat_attention_1,
#                                                 #dropout, trng, drop_ratio)
#         combined_feat_attention_1 = fflayer1(shared_params,
#                                     combined_feat_attention_1, options,
#                                     prefix='combined_att_mlp_1',)
#         #combined_feat_attention_1.shape 1,196,1
        
#         prob_attention_1 = self.softmax(combined_feat_attention_1[:, :, 0])
#         #prob_attention_1.shape 1,196

#         image_feat_ave_1 = (prob_attention_1[:, :, None] * image_feat_down).asnumpy().sum(axis=1)
#         image_feat_ave_1 = Tensor(image_feat_ave_1)
        
#         combined_hidden_1 = image_feat_ave_1 + h_encode
#         #combined_hidden_1 1,1024
        
#         # second layer attention model
#         image_feat_attention_2 = fflayer(shared_params, image_feat_down, options,
#                                      prefix='image_att_mlp_2',
#                                      act_func=options.get('image_att_mlp_act',
#                                                           'tanh'))

#         h_encode_attention_2 = fflayer(shared_params, combined_hidden_1, options,
#                                    prefix='sent_att_mlp_2',
#                                    act_func=options.get('sent_att_mlp_act',
#                                                         'tanh'))
        
#         combined_feat_attention_2 = image_feat_attention_2 + \
#                                 h_encode_attention_2[:, None, :]
        
#         combined_feat_attention_2 = fflayer(shared_params,
#                                         combined_feat_attention_2, options,
#                                         prefix='combined_att_mlp_2',
#                                         act_func=options.get(
#                                             'combined_att_mlp_act', 'tanh'))
        
#         prob_attention_2 = self.softmax(combined_feat_attention_2[:, :, 0])
        
#         image_feat_ave_2 = (prob_attention_2[:, :, None] * image_feat_down).asnumpy().sum(axis=1)
#         image_feat_ave_2 = Tensor(image_feat_ave_2)
        
#         combined_hidden = image_feat_ave_2 + combined_hidden_1
        
#         prob = self.softmax(combined_hidden)
        
        
#         return image_feat

In [63]:
vqa = VQA(options)
ds = create_dataset(options['batch_size'], mode='val',q_dict = question_key,a_dict = answer_top_k)

In [52]:
i = 0
for data in ds.create_dict_iterator():
    i+=1
    if i<5:
        print("question:",type(data['question']))
        print('{}'.format(data['question']),'{}'.format(data['label']))
    else:
        break

question: <class 'mindspore.common.tensor.Tensor'>
[[   4    3    2  129   80    8    0    0    0    0]
 [   3    6   47    7   10  284    0    0    0    0]
 [  11   12  171    0    0    0    0    0    0    0]
 [   4    3   14   19  858    8    0    0    0    0]
 [   4    3    2   16   20    0    0    0    0    0]
 [  11   12  373    5    7    2   27    0    0    0]
 [   4    3    2   44   31    0    0    0    0    0]
 [   3    6   10  393  111    0    0    0    0    0]
 [  11   12  135    5    7    2  240    0    0    0]
 [   3    2   37   10  187    0    0    0    0    0]
 [   5   23   17   40   10  432 1299    0    0    0]
 [   3    2  231  471    0    0    0    0    0    0]
 [ 866  533    3    8    2 1665   64 1600    0    0]
 [   3    2   16  830   50    2   18    0    0    0]
 [   3   14   10 1030    8    2  141    9    2  164]
 [  15    6  172   19   36   10 2926    0    0    0]
 [   5   14   43   39    7    2  645    0    0    0]
 [  15    2   47   79 1311    0    0    0    0  

In [54]:
class MyWithLossCell(nn.Cell):
    def __init__(self, network, loss_fn):
        super(MyWithLossCell, self).__init__(auto_prefix=False)
        self._backbone = network
        self._loss_fn = loss_fn

    def construct(self, image, question, label):
        out = self._backbone(image, question)
        return self._loss_fn(out, label)
        # return out

    @property
    def backbone_network(self):
        return self._backbone

In [64]:
opt = nn.Momentum(params=vqa.trainable_params(), learning_rate=0.1, momentum=0.9)
loss = nn.MSELoss()
net_with_loss = MyWithLossCell(vqa, loss)
net_one_step = nn.TrainOneStepCell(net_with_loss, optimizer=opt)

In [68]:
vqa.trainable_params()

[Parameter (name=dense1.weight, shape=(1000, 1024), dtype=Float32, requires_grad=True),
 Parameter (name=dense1.bias, shape=(1000,), dtype=Float32, requires_grad=True),
 Parameter (name=dense2.weight, shape=(1024, 512), dtype=Float32, requires_grad=True),
 Parameter (name=dense2.bias, shape=(1024,), dtype=Float32, requires_grad=True),
 Parameter (name=question.embedding.embedding_table, shape=(6620, 500), dtype=Float32, requires_grad=True),
 Parameter (name=question.lstm.rnn.weight_fw0, shape=(1524, 4096), dtype=Float16, requires_grad=True),
 Parameter (name=question.lstm.rnn.bias_fw0, shape=(4096,), dtype=Float16, requires_grad=True),
 Parameter (name=vgg.layers.0.weight, shape=(64, 3, 3, 3), dtype=Float32, requires_grad=True),
 Parameter (name=vgg.layers.2.weight, shape=(64, 64, 3, 3), dtype=Float32, requires_grad=True),
 Parameter (name=vgg.layers.5.weight, shape=(128, 64, 3, 3), dtype=Float32, requires_grad=True),
 Parameter (name=vgg.layers.7.weight, shape=(128, 128, 3, 3), dtype=

In [65]:
def train_eval():
    """Train model."""
    train_net = net_one_step
    loss_list = []
    epoch = 0
    
    for data in ds.create_dict_iterator():
        epoch+=1
        if epoch< 5:
            train_net.set_train()
            train_result = train_net(data['image'],data['question'],data['label'])
            print("Epoch:{0}\n loss:{1}:".format(epoch,train_result))
        else:
            break

In [66]:
train_eval()

RuntimeError: mindspore/ccsrc/backend/kernel_compiler/tbe/tbe_kernel_parallel_build.cc:99 TbeOpParallelBuild] task compile Failed, task id:201, cause:TBEException:ERROR:

Traceback (most recent call last):
  File "/home/ma-user/miniconda3/envs/MindSpore-python3.7-aarch64/lib/python3.7/site-packages/mindspore/_extends/parallel_compile/tbe_compiler/compiler.py", line 128, in build_op
    res = op_func(*inputs_args, *outputs_args, *attrs_args, kernel_name=kernel_name)
  File "/usr/local/Ascend/nnae/latest/fwkacllib/python/site-packages/tbe/common/utils/para_check.py", line 529, in _in_wrapper
    return func(*args, **kwargs)
  File "/usr/local/Ascend/nnae/latest/opp/op_impl/built-in/ai_core/tbe/impl/strided_slice_grad_d.py", line 1071, in strided_slice_grad_d
    _check_mask(shrink_axis_mask, True)
  File "/usr/local/Ascend/nnae/latest/opp/op_impl/built-in/ai_core/tbe/impl/strided_slice_grad_d.py", line 977, in _check_mask
    raise RuntimeError("shrink_axis_mask only support {} currently

In [167]:
model = Model(net_one_step)
model.train(1,ds)

RuntimeError: mindspore/ccsrc/backend/session/ascend_session.cc:905 BuildKernel] Kernel build error.

# 

In [1]:
from mindspore import dtype as mstype
test_batch = 8
all_question_idx = Tensor(np.array(all_question_idx),mstype.int32)
x = all_question_idx[0:8,]
#modify
embedding = nn.Embedding(options['dict_size'], options['n_emb'],True)
embeddings = embedding(x)
#这里的8是batch_size
h = Tensor(np.zeros((1,8, options['n_dim']), dtype='float32'))
c = Tensor(np.zeros((1,8, options['n_dim']), dtype='float32'))
        
net = nn.LSTM(options['n_emb'],options['n_dim'],1,True,True)
output,(h1,c1) = net(embeddings, (h,c))
h_encode = h1[0][-1]
h_encode = h_encode[None,:]
# h_encode.shape 1,1024

NameError: name 'Tensor' is not defined

In [33]:
image_feat = fflayer(shared_params, image_feat, options,
                        prefix='image_mlp',
                        act_func='tanh')
image_feat = image_feat[0]
#image_feat.shape 196,1024
        
image_feat_down = image_feat
#image_feat_down.shape 196,1024
        
image_feat_attention_1 = fflayer(shared_params, image_feat_down, options,
                                    prefix='image_att_mlp_1',
                                    act_func='tanh')
        #image_feat_attention_1.shape 196,512

        
h_encode_attention_1 = fflayer(shared_params, h_encode, options,
                                prefix='sent_att_mlp_1',
                                act_func='tanh')
        #h_encode_attentioshapehape 1,512
        
combined_feat_attention_1 = image_feat_attention_1 + \
                    h_encode_attention_1[:, None, :]
        #combined_feat_attention_1.shape 1,196,512

        #if options['use_attention_drop']:
        #combined_feat_attention_1 = dropout_layer(combined_feat_attention_1,
                                                #dropout, trng, drop_ratio)
combined_feat_attention_1 = fflayer1(shared_params,
                            combined_feat_attention_1, options,
                            prefix='combined_att_mlp_1',)
        #combined_feat_attention_1.shape 1,196,1
        
softmax = nn.Softmax()
prob_attention_1 = softmax(combined_feat_attention_1[:, :, 0])
        #prob_attention_1.shape 1,196

image_feat_ave_1 = (prob_attention_1[:, :, None] * image_feat_down).asnumpy().sum(axis=1)
image_feat_ave_1 = Tensor(image_feat_ave_1)
        
combined_hidden_1 = image_feat_ave_1 + h_encode
        #combined_hidden_1 1,1024

In [21]:
def lstm_layer(shared_params, x, mask, h_0, c_0, options, prefix='lstm'):
    ''' lstm layer:
    :param shared_params: shared parameters
    :param x: input, T x batch_size x n_emb
    :param mask: mask for x, T x batch_size
    '''
    n_emb = options['n_emb']
    n_dim = options['n_dim']
    # weight matrix for x, n_emb x 4*n_dim (ifoc)
    lstm_w_x = shared_params[prefix + '_w_x']
    # weight matrix for h, n_dim x 4*n_dim
    lstm_w_h = shared_params[prefix + '_w_h']
    lstm_b_h = shared_params[prefix + '_b_h']
    h_0 = h_0[:x.shape[1]]
    c_0 = c_0[:x.shape[1]]
    question_net = LSTM(n_emb, n_dim)
    output, (h, c) = question_net(x, (h_0, c_0))
    return h, c

In [22]:
def build_model(shared_params, options):
    #input_idx = Tensor.imatrix('input_idx')
    input_idx = Tensor()
    global empty_word
    empty_word = np.zeros((1, options['n_emb']), dtype='float32')
    w_emb_extend = Tensor.concatenate([empty_word, shared_params['w_emb']],
                                 axis=0)
    input_emb = w_emb_extend[input_idx]
    
    # get the transformed image feature
    global h_0, c_0
    h_0 = np.zeros((batch_size, n_dim), dtype='float32')
    c_0 = np.zeros((batch_size, n_dim), dtype='float32')
    h_encode, c_encode = lstm_layer(shared_params, input_emb, input_mask,
                                    h_0, c_0, options, prefix='sent_lstm')
    return h_encodem, c_encode